In [4]:
import numpy as np
import pandas as pd

import os
df = pd.read_excel("Supply chain logisitcs problem.xlsx")


In [5]:
freight = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name = 'FreightRates')
order = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name = 'OrderList')
plant_ports = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name = 'PlantPorts')
prod_per_plant = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name = 'ProductsPerPlant')
vmi_cust = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name = 'VmiCustomers')
wh_cap = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name = 'WhCapacities')
wh_costs = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name = 'WhCosts')

In [6]:
dict(zip(freight['orig_port_cd'],freight['dest_port_cd']))


{'PORT08': 'PORT09',
 'PORT10': 'PORT09',
 'PORT09': 'PORT09',
 'PORT11': 'PORT09',
 'PORT04': 'PORT09',
 'PORT02': 'PORT09',
 'PORT03': 'PORT09',
 'PORT07': 'PORT09',
 'PORT05': 'PORT09',
 'PORT06': 'PORT09'}

In [7]:
freight.isnull().sum()


Carrier         0
orig_port_cd    0
dest_port_cd    0
minm_wgh_qty    0
max_wgh_qty     0
svc_cd          0
minimum cost    0
rate            0
mode_dsc        0
tpt_day_cnt     0
Carrier type    0
dtype: int64

In [8]:
order.isnull().sum()


Order ID                0
Order Date              0
Origin Port             0
Carrier                 0
TPT                     0
Service Level           0
Ship ahead day count    0
Ship Late Day count     0
Customer                0
Product ID              0
Plant Code              0
Destination Port        0
Unit quantity           0
Weight                  0
dtype: int64

In [ ]:
plant_ports.isnull().sum()


Plant Code    0
Port          0
dtype: int64

In [ ]:
prod_per_plant.isnull().sum()


Plant Code    0
Product ID    0
dtype: int64

In [ ]:
vmi_cust.isnull().sum()


Plant Code    0
Customers     0
dtype: int64

In [ ]:
wh_cap.isnull().sum()


Plant ID           0
Daily Capacity     0
dtype: int64

In [ ]:
wh_costs.isnull().sum()


WH           0
Cost/unit    0
dtype: int64

In [30]:
order = order[['Order Date','Order ID','Carrier','Origin Port','Destination Port','Unit quantity','Weight']]

df_merge = pd.merge(order, plant_ports, left_on = ['Origin Port'], right_on = ['Port'])
df_merge = df_merge.rename(columns = {'Plant Code' : 'origin_plant_code'})
df_merge = df_merge.drop(['Port'], axis = 1)
df_merge = pd.merge(df_merge, plant_ports, left_on = ['Destination Port'], right_on = ['Port'])
df_merge = df_merge.drop(['Port'], axis = 1)
df_merge = df_merge.rename(columns = {'Plant Code' : 'destination_plant_code'})
df_merge.head(10)

,Order Date,Order ID,Carrier,Origin Port,Destination Port,Unit quantity,Weight,origin_plant_code,destination_plant_code
0,2013-05-26,1.447296e+09,V44_3,PORT09,PORT09,808,14.30,PLANT16,PLANT16
1,2013-05-26,1.447158e+09,V44_3,PORT09,PORT09,3188,87.94,PLANT16,PLANT16
2,2013-05-26,1.447139e+09,V44_3,PORT09,PORT09,2331,61.20,PLANT16,PLANT16
3,2013-05-26,1.447364e+09,V44_3,PORT09,PORT09,847,16.16,PLANT16,PLANT16
4,2013-05-26,1.447364e+09,V44_3,PORT09,PORT09,2163,52.34,PLANT16,PLANT16
5,2013-05-26,1.447351e+09,V44_3,PORT09,PORT09,3332,92.80,PLANT16,PLANT16
6,2013-05-26,1.447320e+09,V44_3,PORT09,PORT09,1782,46.90,PLANT16,PLANT16
7,2013-05-26,1.447158e+09,V44_3,PORT09,PORT09,427,2.86,PLANT16,PLANT16
8,2013-05-26,1.447219e+09,V44_3,PORT09,PORT09,1291,26.60,PLANT16,PLANT16
9,2013-05-26,1.447398e+09,V44_3,PORT09,PORT09,2294,62.20,PLANT16,PLANT16


In [10]:
freight_new = freight.groupby(['Carrier','orig_port_cd','dest_port_cd']).agg({
    'minm_wgh_qty' : lambda x : sum(x)/len(x),
    'max_wgh_qty' : lambda x : sum(x)/len(x),
    'rate' : lambda x : sum(x)/len(x),
    'minimum cost' : lambda x : sum(x)/len(x)
}).reset_index().rename(columns = {'minimum cost' : 'min_cost','minm_wgh_qty' : 'min_wgh_qty'})

freight_new

,Carrier,orig_port_cd,dest_port_cd,min_wgh_qty,max_wgh_qty,rate,min_cost
0,V444_0,PORT02,PORT09,570.000000,20569.990000,0.084480,23.838400
1,V444_0,PORT03,PORT09,0.000000,5000.000000,12.694200,28.944200
2,V444_0,PORT04,PORT09,570.000000,20569.990000,0.052000,2.477200
3,V444_0,PORT09,PORT09,0.000000,5000.000000,12.778400,31.278400
4,V444_1,PORT02,PORT09,197.556000,1190.745000,0.291140,23.557340
5,V444_1,PORT04,PORT09,197.556000,1190.745000,0.242000,20.909780
6,V444_1,PORT05,PORT09,170.130000,1001.952500,0.457895,35.954091
7,V444_1,PORT06,PORT09,45.473493,111.272697,2.492247,6.917300
8,V444_1,PORT10,PORT09,78.945000,12570.433750,0.431400,12.350850
9,V444_2,PORT02,PORT09,570.000000,20569.990000,0.079040,9.027200


In [11]:
cost = freight_new.groupby(['orig_port_cd','dest_port_cd'])['min_cost'].min().unstack()
cost = round(cost,0)
cost = cost[cost.index.isin(df_merge['Origin Port'])]
cost_pulp = cost.to_numpy().tolist()
cost_pulp

[[2.0], [8.0], [24.0]]

In [12]:
cost


dest_port_cd,PORT09
orig_port_cd,
PORT04,2.0
PORT05,8.0
PORT09,24.0


In [13]:
wh_cap


,Plant ID,Daily Capacity
0,PLANT15,11
1,PLANT17,8
2,PLANT18,111
3,PLANT05,385
4,PLANT02,138
5,PLANT01,1070
6,PLANT06,49
7,PLANT10,118
8,PLANT07,265
9,PLANT14,549


In [14]:
origins = df_merge['Origin Port'].unique().tolist()
origins = sorted(origins)
destinations = df_merge['Destination Port'].unique().tolist()

df_origin = df_merge.groupby(['Origin Port']).agg({
    'Order ID' : 'nunique',
    'Order Date' : 'nunique',
    'Unit quantity' : 'sum',
    'Weight' : 'sum'
}).reset_index()

df_origin['avg_qty'] = round(df_origin['Unit quantity']/df_origin['Order Date'],0)
df_origin['avg_weight'] = round(df_origin['Weight']/df_origin['Order Date'],0)
df_origin = df_origin.drop(['Order ID','Unit quantity','Weight'], axis = 1)


df_dest = df_merge.groupby(['Destination Port']).agg({
    'Order ID' : 'nunique',
    'Order Date' : 'nunique',
    'Unit quantity' : 'sum',
    'Weight' : 'sum'
}).reset_index()

df_dest['avg_qty'] = round(df_dest['Unit quantity']/df_dest['Order Date'],0)
df_dest['avg_weight'] = round(df_dest['Weight']/df_dest['Order Date'],0)
df_dest = df_dest.drop(['Order ID','Unit quantity','Weight'], axis = 1)

df_dest

,Destination Port,Order Date,avg_qty,avg_weight
0,PORT09,1,205120319.0,1088803.0


In [15]:
df_origin


,Origin Port,Order Date,avg_qty,avg_weight
0,PORT04,1,204874838.0,1056632.0
1,PORT05,1,348.0,2.0
2,PORT09,1,245133.0,32168.0


In [16]:
supply = dict(zip(df_origin['Origin Port'],df_origin['avg_weight']))
demand = dict(zip(df_dest['Destination Port'], df_dest['avg_weight']))

#All Variables for Routing Optimization
# supply['PORT04'] = 1013
# supply['PORT05'] = 385
# supply['PORT09'] = 457
# demand['PORT09'] = 457
print(f'All Origin Nodes : {origins}')
print(f'Origin Supply : {supply}')
print(f'All Destination Nodes : {destinations}')
print(f'Destination Demand : {demand}')
print(f'Costs From Origin to Destination : {cost_pulp}')

All Origin Nodes : ['PORT04', 'PORT05', 'PORT09']
Origin Supply : {'PORT04': 1056632.0, 'PORT05': 2.0, 'PORT09': 32168.0}
All Destination Nodes : ['PORT09']
Destination Demand : {'PORT09': 1088803.0}
Costs From Origin to Destination : [[2.0], [8.0], [24.0]]


In [17]:

!pip install pulp
from pulp import *

costs = makeDict([origins,destinations], cost_pulp, 0)
prob = LpProblem("Port Distribution Problem", LpMinimize)

routes = [(o,d) for o in origins for d in destinations]

vars = LpVariable.dicts("route", (origins,destinations), 0 , None, LpInteger)

prob += (
lpSum([vars[o][d] * costs[o][d] for (o,d) in routes]),
"Sum of Transporting Costs"
)

prob

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Port_Distribution_Problem:
MINIMIZE
2.0*route_PORT04_PORT09 + 8.0*route_PORT05_PORT09 + 24.0*route_PORT09_PORT09 + 0.0
VARIABLES
0 <= route_PORT04_PORT09 Integer
0 <= route_PORT05_PORT09 Integer
0 <= route_PORT09_PORT09 Integer

In [18]:
for o in origins :
    prob += (
    lpSum([vars[o][d] for d in destinations]) <= supply[o],
        "Sum_of_Weight_of_Goods_out_of_Origin_Ports_%s" % o,
    )

for d in destinations :
    prob += (
    lpSum([vars[o][d] for o in origins]) >= demand[d],
        "Sum_of_Weight_of_Goods_into_Destination_Ports_%s" % d,
    )

In [19]:
prob.solve()
for v in prob.variables() :
    print(f'{v} : {v.varValue}')

route_PORT04_PORT09 : 1056632.0
route_PORT05_PORT09 : 2.0
route_PORT09_PORT09 : 32169.0


VERSION 2

In [20]:
cost_2 = freight.groupby(['orig_port_cd','dest_port_cd']).agg({
    'minimum cost' : lambda x : round((x.max() + x.min())/2,0)
}).unstack()

cost_pulp_2 = cost_2['minimum cost'].to_numpy().tolist()
cost_pulp_2

[[60.0],
 [222.0],
 [52.0],
 [107.0],
 [60.0],
 [16.0],
 [125.0],
 [24.0],
 [33.0],
 [18.0]]

In [21]:
plant_ports_filtered = plant_ports.set_index('Port')
plant_ports_filtered = plant_ports_filtered[plant_ports_filtered.index.isin(cost_2.index)]
pp_filtered = plant_ports_filtered.reset_index().sort_values(by = 'Port')
pp_filtered

,Port,Plant Code
0,PORT02,PLANT01
6,PORT02,PLANT07
9,PORT02,PLANT10
1,PORT03,PLANT02
7,PORT04,PLANT08
8,PORT04,PLANT09
10,PORT04,PLANT11
11,PORT04,PLANT12
12,PORT04,PLANT13
2,PORT04,PLANT03


In [22]:
pp_merged = pd.merge(pp_filtered, wh_cap, left_on = ['Plant Code'], right_on = ['Plant ID'], how = 'left')
pp_merged = pp_merged.drop(['Plant ID'], axis = 1)
pp_sum = pp_merged.groupby(['Port'])['Daily Capacity '].sum().reset_index()
pp_sum

,Port,Daily Capacity
0,PORT02,1453
1,PORT03,138
2,PORT04,2076
3,PORT05,554
4,PORT06,434
5,PORT07,549
6,PORT08,11
7,PORT09,457
8,PORT10,8
9,PORT11,111


In [23]:
perc_needed = pp_sum[pp_sum['Port'] == 'PORT09']['Daily Capacity ']/sum(pp_sum['Daily Capacity '])
perc_needed

7    0.078916
Name: Daily Capacity , dtype: float64

In [24]:
pp_merged.columns

Index(['Port', 'Plant Code', 'Daily Capacity '], dtype='object')

In [25]:
f_origin = freight[['orig_port_cd']].drop_duplicates().sort_values(by = 'orig_port_cd')
f_dest = freight[['dest_port_cd']].drop_duplicates().sort_values(by = 'dest_port_cd')

f_origin = pd.merge(f_origin, pp_sum, left_on = 'orig_port_cd', right_on = 'Port', how = 'left')
f_origin = f_origin[['orig_port_cd','Daily Capacity ']]

f_origin_list = f_origin['orig_port_cd'].tolist()
f_origin_list

['PORT02',
 'PORT03',
 'PORT04',
 'PORT05',
 'PORT06',
 'PORT07',
 'PORT08',
 'PORT09',
 'PORT10',
 'PORT11']

In [26]:
f_dest = pd.merge(f_dest, pp_sum, left_on = 'dest_port_cd', right_on = 'Port', how = 'left')
f_dest = f_dest[['dest_port_cd','Daily Capacity ']]
f_dest

f_dest_list = f_dest['dest_port_cd'].tolist()
f_dest_list

['PORT09']

In [27]:
origin_cap = dict(zip(f_origin['orig_port_cd'], f_origin['Daily Capacity '].astype('float')))
dest_cap = dict(zip(f_dest['dest_port_cd'], f_dest['Daily Capacity '].astype('float')))

origin_cap

{'PORT02': 1453.0,
 'PORT03': 138.0,
 'PORT04': 2076.0,
 'PORT05': 554.0,
 'PORT06': 434.0,
 'PORT07': 549.0,
 'PORT08': 11.0,
 'PORT09': 457.0,
 'PORT10': 8.0,
 'PORT11': 111.0}

In [28]:
dest_cap

{'PORT09': 457.0}

In [29]:
problem_statement = LpProblem("Port Distribution Problem", LpMinimize)

variables = LpVariable.dicts("route", (f_origin_list,f_dest_list), 0 , None, LpInteger)

routes = [(o,d) for o in f_origin_list for d in f_dest_list]

costs = makeDict([f_origin_list,f_dest_list], cost_pulp_2, 0)

problem_statement += (
lpSum([variables[o][d] * costs[o][d] for (o,d) in routes]),
    "Sum_of_Transporting_Costs"
)


for o in f_origin_list :
    problem_statement += (
    lpSum([variables[o][d] for d in f_dest_list]) <= origin_cap[o],
        "Sum_of_Weight_out_of_Port_%s" %o,
    )

for d in f_dest_list :
    problem_statement += (
    lpSum([variables[o][d] for o in f_origin_list]) >= dest_cap[d],
        "Sum_of_Weight_in_Port_%s" %d
    )

problem_statement.solve()

for a in problem_statement.variables():
    print(f'{a} : {a.varValue}')

route_PORT02_PORT09 : 0.0
route_PORT03_PORT09 : 0.0
route_PORT04_PORT09 : 0.0
route_PORT05_PORT09 : 0.0
route_PORT06_PORT09 : 0.0
route_PORT07_PORT09 : 457.0
route_PORT08_PORT09 : 0.0
route_PORT09_PORT09 : 0.0
route_PORT10_PORT09 : 0.0
route_PORT11_PORT09 : 0.0


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
